In [4]:
# conda activate protein-vec
# TODO: this has path issues with the conda env

import torch
from model_protein_moe import trans_basic_block, trans_basic_block_Config
from utils_search import *
from transformers import T5EncoderModel, T5Tokenizer
import re
import gc
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from collections import defaultdict
import faiss


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

ModuleNotFoundError: No module named 'model_protein_moe'

In [ ]:
#Protein-Vec MOE model checkpoint and config
vec_model_cpnt = 'protein_vec_models/protein_vec.ckpt'
vec_model_config = 'protein_vec_models/protein_vec_params.json'

In [3]:
#Load the ProtTrans model and ProtTrans tokenizer
tokenizer = T5Tokenizer.from_pretrained("Rostlab/prot_t5_xl_uniref50", do_lower_case=False )
model = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_uniref50")
gc.collect()

model = model.to(device)
model = model.eval()

Some weights of the model checkpoint at Rostlab/prot_t5_xl_uniref50 were not used when initializing T5EncoderModel: ['decoder.block.10.layer.2.DenseReluDense.wo.weight', 'decoder.block.15.layer.1.EncDecAttention.q.weight', 'decoder.block.0.layer.1.EncDecAttention.q.weight', 'decoder.block.14.layer.1.EncDecAttention.o.weight', 'decoder.block.7.layer.0.SelfAttention.v.weight', 'decoder.block.20.layer.0.SelfAttention.q.weight', 'decoder.block.5.layer.2.DenseReluDense.wi.weight', 'decoder.block.3.layer.0.SelfAttention.v.weight', 'decoder.block.2.layer.2.DenseReluDense.wi.weight', 'decoder.block.5.layer.1.EncDecAttention.q.weight', 'decoder.block.17.layer.1.layer_norm.weight', 'decoder.block.4.layer.1.EncDecAttention.k.weight', 'decoder.block.23.layer.0.SelfAttention.k.weight', 'decoder.block.14.layer.0.SelfAttention.k.weight', 'decoder.block.2.layer.1.EncDecAttention.v.weight', 'decoder.block.5.layer.1.EncDecAttention.k.weight', 'decoder.block.3.layer.0.SelfAttention.o.weight', 'decoder.bl

In [4]:
#Load the model
vec_model_config = trans_basic_block_Config.from_json(vec_model_config)
model_deep = trans_basic_block.load_from_checkpoint(vec_model_cpnt, config=vec_model_config)
model_deep = model_deep.to(device)
model_deep = model_deep.eval()

Lightning automatically upgraded your loaded checkpoint from v1.8.0rc0 to v1.9.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file protein_vec_models/tm_vec_swiss_model_large.ckpt`


In [5]:
# Load in uniprot meta data
meta_data_new = pd.read_csv('data/uniprotkb_AND_reviewed_true_2023_07_03.tsv', sep='\t')

In [6]:
#Now filter for the proteins that were newly discovered
new_proteins = meta_data_new[meta_data_new['Date of creation'] > '2022-05-25'].reset_index(drop=True)

In [7]:
print('Number of new proteins deposited after 2022-05-25')
print(len(new_proteins))

# filter those proteins that are greater than 2000 amino acid residues long
new_proteins['length'] = new_proteins['Sequence'].str.len()
new_proteins = new_proteins[new_proteins['length'] <= 2000]
print('Filtered proteins longer than 2000 amino acids')
print(len(new_proteins))

Number of new proteins deposited after 2022-05-25
2350
Filtered proteins longer than 2000 amino acids
2295


In [8]:
# This is a forward pass of the Protein-Vec model
# Every aspect is turned on (therefore no masks)
sampled_keys = np.array(['TM', 'PFAM', 'GENE3D', 'ENZYME', 'MFO', 'BPO', 'CCO'])
all_cols = np.array(['TM', 'PFAM', 'GENE3D', 'ENZYME', 'MFO', 'BPO', 'CCO'])
masks = [all_cols[k] in sampled_keys for k in range(len(all_cols))]
masks = torch.logical_not(torch.tensor(masks, dtype=torch.bool))[None,:]

#Pull out sequences for the new proteins
flat_seqs = new_proteins['Sequence'].values

#Loop through the sequences and embed them using protein-vec
i = 0
embed_all_sequences = []
while i < len(flat_seqs): 
    protrans_sequence = featurize_prottrans(flat_seqs[i:i+1], model, tokenizer, device)
    embedded_sequence = embed_vec(protrans_sequence, model_deep, masks, device)
    embed_all_sequences.append(embedded_sequence)
    i = i + 1
    if i % 50 == 0:
        print(i)

/home/ron/miniconda3/envs/protein-vec/lib/python3.7/site-packages/torch/nn/modules/transformer.py:276: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:175.)
  output = torch._nested_tensor_from_mask(output, src_key_padding_mask.logical_not(), mask_check=False)


50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250


In [9]:
#Combine the embedding vectors into an array
query_embeddings = np.concatenate(embed_all_sequences)

In [10]:
# save the query embeddings
np.save('new_protein_embeddings.npy', query_embeddings)

In [8]:
query_embeddings = np.load('new_protein_embeddings.npy')

In [9]:
################## Load the lookup database of all embeddings (note that we will pull out only embeddings from proteins that were trained on)
embeddings = np.load('protein_vec_embeddings/lookup_embeddings.npy')
lookup_proteins_meta = pd.read_csv('protein_vec_embeddings/lookup_embeddings_meta_data.tsv', sep="\t")

In [10]:
len(lookup_proteins_meta)

540560

In [11]:
# see if Pfam column is empty for all proteins in new_proteins
np.sum(new_proteins['Pfam'].isna())

431

In [12]:
#Switch this for whichever aspect you want to perform search for
############### User parameter
column = 'Pfam'

In [13]:
# Filter for lookup proteins with annotations for the relavant aspect (don't want to transfer null annotations)
col_lookup = lookup_proteins_meta[~lookup_proteins_meta[column].isnull()]
col_lookup_embeddings = embeddings[col_lookup.index]
col_meta_data = col_lookup[column].values

# load database
lookup_database = load_database(col_lookup_embeddings)

# Query for the 1st nearest neighbor
k = 10
D, I = query(lookup_database, query_embeddings, k)

#Get metadata for the 1st nearest neighbor
near_ids = []
for i in range(I.shape[0]):
    meta = col_meta_data[I[i]]
    near_ids.append(list(meta))       

near_ids = np.array(near_ids)

In [14]:
I.shape

(2295, 10)

In [15]:
I[0]

array([508763, 245040, 367297, 225807, 170500, 326570, 209248, 367880,
       218409, 209249])

In [17]:
meta = col_lookup.iloc[I[0]]

In [22]:
meta["distance"] = D[0]

/home/ron/miniconda3/envs/protein-vec/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [21]:
meta["query_index"] = [0 for i in range(len(meta))]

/home/ron/miniconda3/envs/protein-vec/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [23]:
meta

,Unnamed: 0,Entry,Reviewed,Entry Name,Protein names,Gene Names,Organism,Length,Gene Ontology (biological process),Gene Ontology (cellular component),Gene Ontology (molecular function),Gene Ontology (GO),EC number,Sequence,Date of creation,Gene3D,Pfam,query_index,distance
530860,530860,Q9UT45,reviewed,ENG1_SCHPO,"Primary septum glucan endo-1,3-beta-D-glucosid...",eng1 SPAC821.09,Schizosaccharomyces pombe (strain 972 / ATCC 2...,1016,cell cycle [GO:0007049]; cell wall polysacchar...,cell surface [GO:0009986]; division septum [GO...,"glucan endo-1,3-beta-D-glucosidase activity [G...",cell surface [GO:0009986]; division septum [GO...,3.2.1.39,MSSYLRSFIFGLLTISLAQCSPILKDTKDTKFSTGSNISLKKRDTN...,2004-03-29,1.10.287.1170;1.20.5.420;,PF10645;PF17652;PF03639;,0,0.999972
255285,255285,P53753,reviewed,ENG1_YEAST,"Glucan endo-1,3-beta-D-glucosidase 1 (Endo-1,3...",DSE4 ENG1 YNR067C N3547,Saccharomyces cerevisiae (strain ATCC 204508 /...,1117,cell wall organization [GO:0071555]; polysacch...,cell septum [GO:0030428]; cell surface [GO:000...,"glucan endo-1,3-beta-D-glucosidase activity [G...",cell septum [GO:0030428]; cell surface [GO:000...,3.2.1.39,MQLYLTLLFLLSFVECSYISFISNNADEILETDLIETLSYATLTVG...,1996-10-01,1.20.5.420;,PF17652;PF03639;,0,0.999969
384236,384236,Q5AIR7,reviewed,ENG1_CANAL,"Glucan endo-1,3-beta-D-glucosidase 1 (Endo-1,3...",ENG1 ACF3 DSE4 CAALFM_C103680WA CaO19.10584 Ca...,Candida albicans (strain SC5314 / ATCC MYA-287...,1145,cell wall organization [GO:0071555]; polysacch...,cell septum [GO:0030428]; cell surface [GO:000...,"glucan endo-1,3-beta-D-glucosidase activity [G...",cell septum [GO:0030428]; cell surface [GO:000...,3.2.1.39,MLFKSVLLSTLIAVQALAENSAHQEIVTVTKTTHISNPCLENYAKK...,2014-07-09,1.10.287.1170;1.20.5.420;,PF17652;PF03639;,0,0.999966
234761,234761,P29019,reviewed,GUN_BACSZ,Endoglucanase (EC 3.2.1.4) (Cellulase) (Endo-1...,NaN,Bacillus sp. (strain KSM-330),463,cellulose catabolic process [GO:0030245],NaN,cellulase activity [GO:0008810],cellulase activity [GO:0008810]; cellulose cat...,3.2.1.4,MVEKRKIFTVLCACGIGFTSYTSCISAAAIDNDTLINNGHKINSSI...,1992-12-01,1.50.10.10;,PF01270;,0,0.999928
174689,174689,E9CXX8,reviewed,MNS1B_COCPS,"Mannosyl-oligosaccharide alpha-1,2-mannosidase...",CPSG_02648,Coccidioides posadasii (strain RMSCC 757 / Sil...,519,carbohydrate metabolic process [GO:0005975]; p...,extracellular region [GO:0005576]; membrane [G...,calcium ion binding [GO:0005509]; mannosyl-oli...,extracellular region [GO:0005576]; membrane [G...,3.2.1.113,MKGSPVLAVCAAALTLIPSVVALPMIDKDLPSSISQSSDKTSQERA...,2015-04-29,1.50.10.10;,PF01532;,0,0.999926
341946,341946,Q2U4L7,reviewed,GTAA_ASPOR,Glutaminase A (EC 3.5.1.2),gtaA AO090020000289,Aspergillus oryzae (strain ATCC 42149 / RIB 40...,690,carbohydrate metabolic process [GO:0005975],extracellular region [GO:0005576],glutaminase activity [GO:0004359],extracellular region [GO:0005576]; glutaminase...,3.5.1.2,MMHFLSFCLSVASLVSYAGAASTFSPARPPALPLAVKSPYLSTWLS...,2015-12-09,NaN,PF08760;PF16335;PF17168;,0,0.999924
216385,216385,P0CZ00,reviewed,HYSA_CUTAC,Hyaluronate lyase (EC 4.2.2.1) (Hyaluronidase)...,NaN,Cutibacterium acnes (Propionibacterium acnes),812,carbohydrate metabolic process [GO:0005975],extracellular region [GO:0005576],carbohydrate binding [GO:0030246]; hyaluronate...,extracellular region [GO:0005576]; carbohydrat...,4.2.2.1,MFGTPSRRTFLTASALSAMALAASPTVTDAIAAPGPDSWSALCERW...,2011-06-28,2.70.98.10;1.50.10.100;2.60.220.10;,PF02278;PF08124;,0,0.999923
384873,384873,Q5BF93,reviewed,MNS1B_EMENI,"Mannosyl-oligosaccharide alpha-1,2-mannosidase...",mns1B msdS AN0787,Emericella nidulans (strain FGSC A4 / ATCC 381...,505,carbohydrate metabolic process [GO:0005975]; N...,cytoplasmic vesicle lumen [GO:0060205]; endopl...,calcium ion binding [GO:0005509]; mannosyl-oli...,cytoplasmic vesicle lumen [GO:0060205]; endopl...,3.2.1.113,MRTLLALAALAGFAAARVPAYAITRPVMRSDSRADAVKEAFSHAWD...,2010-06-15,1.50.10.10;,PF01532;,0,0.999922
226844,226844,P19254,reviewed,GUB_NIACI,"Beta-glucanase (EC 3.2.1.

In [16]:
col_meta_data[I[0]]

array(['PF10645;PF17652;PF03639;', 'PF17652;PF03639;', 'PF17652;PF03639;',
       'PF01270;', 'PF01532;', 'PF08760;PF16335;PF17168;',
       'PF02278;PF08124;', 'PF01532;', 'PF01270;',
       'PF02278;PF02884;PF08124;'], dtype=object)

In [42]:
meta_query

'PF07380;'

In [52]:
test[0].split(';')

['PF07380', ' PF04596', '']

In [44]:
test = ['PF07380; PF04596;']

In [43]:
col_meta_data[I[i]]

array(['PF07380;', 'PF07380;', 'PF04596;', 'PF04596;', 'PF05385;',
       'PF04596;', 'PF04776;', 'PF04596;', 'PF03396;', 'PF01104;',
       'PF17378;', 'PF01104;', 'PF03113;', 'PF04596;', 'PF02711;',
       'PF10705;', 'PF01104;', 'PF00803;', 'PF03113;', 'PF07346;',
       'PF04866;', 'PF04866;', 'PF17495;', 'PF19186;', 'PF00803;',
       'PF01677;', 'PF01840;', 'PF07349;', 'PF02711;', 'PF05846;',
       'PF02711;', 'PF04866;', 'PF05385;', 'PF01104;', 'PF01104;',
       'PF00803;', 'PF17467;', 'PF17467;', 'PF01492;', 'PF02711;',
       'PF03025;', 'PF12133;', 'PF16093;', 'PF05864;', 'PF01492;',
       'PF10256;', 'PF01671;', 'PF17575;', 'PF17467;', 'PF08550;',
       'PF01840;', 'PF17378;', 'PF01671;', 'PF04866;', 'PF10705;',
       'PF10705;', 'PF10705;', 'PF05112;', 'PF11986;', 'PF07234;',
       'PF05288;', 'PF01840;', 'PF03338;', 'PF01671;', 'PF08591;',
       'PF04318;', 'PF03113;', 'PF17571;', 'PF04714;', 'PF10256;',
       'PF07349;', 'PF02725;', 'PF04776;', 'PF10718;', 'PF1506

In [37]:
meta = col_meta_data[I[i]]

In [61]:
#Get metadata for the 1st nearest neighbor
near_ids = []
for i in range(I.shape[0]):
    meta = col_meta_data[I[i]]
    meta_query = new_proteins.iloc[i][column]
    # if no annotation for the query, skip
    
    if meta_query is np.nan:
        continue
    # create boolean mask for proteins that have the same annotation as the query
    # print(meta_query)
    mask_exact = [meta_query in meta[k] for k in range(len(meta))]
    # partial hits
    if meta_query.count(';') > 1:
        available_pfs = meta_query.split(';')
        mask_partial = [[available_pfs[j] in meta[k] for j in range(len(available_pfs) - 1)] for k in range(len(meta))]
    near_ids.append({
        'meta': meta,
        'meta_query': meta_query,
        'exact': mask_exact,
        'partial': mask_partial,
        'D_i': D[i]
    })   

# near_ids = np.array(near_ids)

In [78]:
sum(['PF14306' in near_ids[1]['meta'][k] for k in range(len(near_ids[0]['meta']))])

126

In [74]:
near_ids[1]['meta_query']

'PF01583;PF01747;PF14306;'

In [79]:
near_ids[1]['partial'][70]

[False, True, True]

In [64]:
len(near_ids[0]['meta_query'])

16

In [32]:
len(near_ids[0])

1000

In [16]:
near_ids_full = []
for i in range(I.shape[0]):
    meta = col_lookup.iloc[I[i]]
    near_ids_full.append(meta)

In [17]:
df_near_ids_full = pd.DataFrame(columns=near_ids_full[0].columns)

# Append each Series to the DataFrame
for series in near_ids_full:
    df_near_ids_full = df_near_ids_full.append(series, ignore_index=True)

In [19]:
I.shape

(2295, 1000)

In [18]:
D.shape

(2295, 1000)

In [22]:
# flatten D so that the distances are in a single column
D_flat = D.flatten()

In [23]:
D[0, 1]

0.9999694

In [25]:
D_flat[1]

0.9999694

In [26]:
# in reality, we also need the D score for each of the nearest neighbors
df_near_ids_full['D'] = D_flat

In [53]:
df_near_ids_full.iloc[2]['Pfam']

'PF17652;PF03639;'

In [51]:
# check if multiple pfams in the same protein
df_near_ids_full.iloc[3]['Pfam'].count(';')

1